In [1]:
import io

import pandas as pd
from Bio import Entrez, SeqIO
from mysql.connector import connection
from sshtunnel import SSHTunnelForwarder

# Укажите ваш email (обязательно для использования Entrez)
Entrez.email = "your.email@example.com"

In [2]:
with open("db_curated_server_info.txt", "r") as file:
    lines = file.readlines()

config = {}

for line in lines:
    line = line.strip()
    if line and not line.startswith("#"):
        key, value = line.split("=", 1)
        config[key] = value.strip()

server_name = config.get("server_name")
srever_port = int(config.get("srever_port"))
ssh_password = config.get("ssh_password")
ssh_username = config.get("ssh_username")
db_adress = config.get("db_adress")
db_port = int(config.get("db_port"))

In [3]:
tunnel = SSHTunnelForwarder(
    (server_name, srever_port),
    ssh_password=ssh_password,
    ssh_username=ssh_username,
    remote_bind_address=(db_adress, db_port),
)
tunnel.start()
print(tunnel.local_bind_port)

44147


In [4]:
conn = connection.MySQLConnection(
    user="db_user",
    password="db_password",
    host="localhost",
    port=tunnel.local_bind_port,
    database="db_name",
)
cursor = conn.cursor()

In [5]:
query = "SHOW TABLES;"
cursor.execute(query)
cursor.fetchall()

[('alternative_name',),
 ('histone',),
 ('histone_description',),
 ('histone_has_publication',),
 ('publication',),
 ('sequence',),
 ('sequence_has_publication',)]

In [9]:
# add_histone = (
#     "INSERT INTO histone "
#     "(id, level, taxonomic_span, taxonomic_span_id, description, parent) "
#     "VALUES (%(id)s, %(level)s, %(taxonomic_span)s, %(taxonomic_span_id)s, %(description)s, %(parent)s)"
# )
# add_histone_description = (
#     "INSERT INTO histone_description "
#     "(summary, taxonomy, genes, evolution, expression, knock_out, function, sequence, localization, deposition, structure, interactions, disease, caveats) "
#     "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
# )
add_publication = (
    "INSERT INTO publication "
    "(id, title, doi, author, year) "
    "VALUES (%(id)s, %(title)s, %(doi)s, %(author)s, %(year)s)"
)
# add_sequence = (
#     "INSERT INTO sequence "
#     "(accession, variant, gi, ncbi_gene_id, hgnc_gene_name, taxonomy_id, organism, phylum, class, taxonomy_group, info, sequence, variant_under_consideration) "
#     "VALUES (%(accession)s, %(variant)s, %(gi)s, %(ncbi_gene_id)s, %(hgnc_gene_name)s, %(taxonomy_id)s, %(organism)s, %(phylum)s, %(class)s, %(taxonomy_group)s, %(info)s, %(sequence)s, %(variant_under_consideration)s)"
# )
# add_sequence_has_publication = (
#     "INSERT INTO sequence_has_publication "
#     "(sequence_accession, publication_id) "
#     "VALUES (%s, %s)"
# )
add_alternate_names = (
    "INSERT INTO alternative_name "
    "(name, taxonomy, gene, splice, histone) "
    "VALUES (%(name)s, %(taxonomy)s, %(gene)s, %(splice)s, %(histone)s)"
)
add_histone_has_publication = (
    "INSERT INTO histone_has_publication "
    "(histone_id, publication_id) "
    "VALUES (%s, %s)"
)

In [ ]:
data_histone_description = {
    "summary": None,
    "taxonomy": None,
    "genes": None,
    "evolution": None, 
    "expression": None,
    "knock_out": None,
    "function": None,
    "sequence": None,
    "localization": None,
    "deposition": None,
    "structure": None,
    "interactions": None,
    "disease": None,
    "caveats": None,
}

In [10]:
def get_taxonomy_data(record):
    import re
    import sys

    taxonomy_data = {}
    taxonomy_data["organism"] = record.annotations["organism"]
    try:
        for a in record.features[0].qualifiers["db_xref"]:
            text = re.search("(\S+):(\S+)", a).group(1)
            taxid = re.search("(\S+):(\S+)", a).group(2)
            if text == "taxon":
                print(f"Fetched taxid from NCBI {taxid}")
                taxonomy_data["taxonomy_id"] = int(taxid)
            else:
                continue
    except:
        print("!!!!!!Unable to get TAXID for this record setting it to 1")
        taxonomy_data["taxonomy_id"] = 1  # unable to identify

    lineage = dict()
    for i in range(10):
        try:
            handle = Entrez.efetch(
                id=taxonomy_data["taxonomy_id"], db="taxonomy", retmode="xml"
            )
            tax_data = Entrez.read(handle)
            lineage = {
                d["Rank"]: d["ScientificName"]
                for d in tax_data[0]["LineageEx"]
                if d["Rank"] in ["class", "phylum"]
            }
            break
        except:
            print(
                "Unexpected error: {}, Retrying, attempt {}".format(
                    sys.exc_info()[0], i
                )
            )
            if i == 9:
                print(
                    f"FATAL ERROR could not get class and phylum from NCBI after 10 attempts for taxid:{taxonomy_data['taxonomy_id']}. Will add None for class and phylum!"
                )
            else:
                continue
    taxonomy_data["phylum"] = lineage.get("phylum", None)
    taxonomy_data["class"] = lineage.get("class", None)
    if taxonomy_data["phylum"] is not None:
        taxonomy_data["phylum"] = str(taxonomy_data["phylum"])
    if taxonomy_data["class"] is not None:
        taxonomy_data["class"] = str(taxonomy_data["class"])
    return taxonomy_data

In [13]:
query = "SELECT * FROM sequence " "WHERE variant LIKE 'H3%'"
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df["variant"].value_counts()

variant
H3-like_(Viruses)        45
H3.3                     18
H3.Y_(Primates)           4
H3.3_(Homo_sapiens)       2
H3.4_(Homo_sapiens)       1
H3.4_(Mammalia)           1
H3.5_(Hominidae)          1
H3.5_(Homo_sapiens)       1
H3.Y.1_(Homo_sapiens)     1
H3.Y.2_(Homo_sapiens)     1
Name: count, dtype: int64

In [6]:
query = "SELECT * FROM sequence " "WHERE variant='H3.Y_(Primates)'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,accession,variant,gi,ncbi_gene_id,hgnc_gene_name,taxonomy_id,organism,phylum,class,taxonomy_group,info,sequence,variant_under_consideration
0,HISTDB_H3_Y_0,H3.Y_(Primates),NOGI,None,None,9544,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAAAKRAPPRGGIKKPHRYKPGTQ...,None
1,HISTDB_H3_Y_1,H3.Y_(Primates),NOGI,None,None,9544,Macaca mulatta,Chordata,Mammalia,None,None,ARTKQTARKATNWQAPRKPLATKAPGKRLPPRGGIKKPHRYRPGTQ...,None
2,HISTDB_H3_Y_2,H3.Y_(Primates),NOGI,None,None,9598,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAAGKRAPPTGGIKKPHRYKPGTL...,None
3,HISTDB_H3_Y_3,H3.Y_(Primates),NOGI,None,None,9598,Pan troglodytes,Chordata,Mammalia,None,None,ARTKQTARKATAWQAPRKPLATKAARKRASPTGGIKKPHRYKPGTL...,None


In [8]:
query = "SELECT * FROM alternative_name " "WHERE name='cid'"
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,id,name,taxonomy,gene,splice,histone
0,65,cid,None,None,None,cenH3_(Plants)


In [14]:
query = "SELECT * FROM publication "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["id"].str.startswith("hara")]

,id,title,doi,author,year,pubmed_id


In [85]:
query = (
    "SELECT * FROM histone as h LEFT JOIN histone_description as hd "
    "ON h.description = hd.id "
    # "WHERE h.id = 'H3.6_(Homo_sapiens)' "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df.iloc[:,0].str.contains("H3.6")]

,id,level,taxonomic_span,taxonomic_span_id,description,parent,id,summary,taxonomy,genes,...,knock_out,function,sequence,localization,deposition,structure,interactions,disease,caveats,relations
17,cenH3.6_(Repleta),variant,repleta group,32321,245.0,cenH3_(Drosophilidae),245.0,cenH3.6_(Repleta) is a recently evolved centr...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [28]:
df["genes"].values[0]

'CENP-A nucleosomes are essential for chromosome segregation. A functional kinetochore interacts with active centromeric chromatin reached with CENP-A to form the mitotic spindle. Notably, in certain holocentric organisms, CENP-A appears dispensable for meiotic chromosome segregation while remaining essential for mitosis. As demonstrated in Caenorhabditis elegans, CENP-A loading is specifically eliminated following meiosis I, and RNAi-mediated depletion of CENP-A during meiosis fails to disrupt proper chromosome segregation [monen_differential_2005]. This stands in striking contrast to mitotic divisions, where CENP-A is absolutely required for kinetochore assembly and faithful chromosome segregation. Moreover, C. elegans and C. remanei possess a second cenH3 histone gene that likely performs specialized functions distinct from canonical CENP-A roles [monen_separase_2015]. For more detailed information see the "gene" section.'

In [70]:
query = "SELECT * FROM histone_description "
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
# df
for s in df.apply(lambda row: ''.join(row.astype(str)), axis=1):
    if "henikoff_heterochromatic_2000" in s:
        print(s)
        print("---------------")

In [41]:
query = "SHOW TABLES; "
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

,Tables_in_db_name
0,alternative_name
1,histone
2,histone_description
3,histone_has_publication
4,publication
5,sequence
6,sequence_has_publication


In [11]:
query = (
    "SELECT s.accession, s.variant, s.organism, p.* FROM sequence s LEFT JOIN "
    "sequence_has_publication sp ON s.accession = sp.sequence_accession LEFT JOIN "
    "publication p ON sp.publication_id = p.id "
)
cursor.execute(query)
df = pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])
df[df["variant"].str.startswith("H2A.Q", na=False)]

,accession,variant,organism,id,title,doi,author,year,pubmed_id
1407,ELK02218.1,H2A.Q_(Eutheria),Pteropus alecto,29549088,None,None,None,None,None
2367,HISTDB_H2A_Q_0,H2A.Q_(Eutheria),Cercopithecus aethiops,29549088,None,None,None,None,None
2368,HISTDB_H2A_Q_1,H2A.Q_(Eutheria),Pantholops hodgsonii,29549088,None,None,None,None,None
2369,HISTDB_H2A_Q_10,H2A.Q_(Eutheria),Ursus maritimus,29549088,None,None,None,None,None
2370,HISTDB_H2A_Q_11,H2A.Q_(Eutheria),Cercocebus atys,29549088,None,None,None,None,None
2371,HISTDB_H2A_Q_12,H2A.Q_(Eutheria),Odobenus rosmarus divergens,29549088,None,None,None,None,None
2372,HISTDB_H2A_Q_13,H2A.Q_(Eutheria),Ceratotherium simum,29549088,None,None,None,None,None
2373,HISTDB_H2A_Q_14,H2A.Q_(Eutheria),Bos mutus,29549088,None,None,None,None,None
2374,HISTDB_H2A_Q_15,H2A.Q_(Eutheria),Pan troglodytes,29549088,None,None,None,None,None
2375,HISTDB_H2A_Q_16,H2A.Q_(Eutheria),Bos taurus,29549088,None,None,None,None,None


# To Do H1

## <span style="color:black">Update description of cH1.1_(Homo_sapiens)</span>

### <span style="color:black">Update summary</span>

```cH1.1_(Homo sapiens) is a replication-dependent linker histone in humans that is essential during the early stages of embryogenesis [funaya_involvement_2023]. Human cH1.1 interacts with the phosphorylated form of the recombinant protein PC4 to mediate chromatin condensation [mustafi_phosphorylation-dependent_2022].```



# To Do H3



# To Do H4


# Close connections

In [35]:
cursor.close()
conn.close()
tunnel.stop()